In [ ]:
import cv2
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use('TkAgg')  # Ensure GUI popup works

# --- Config ---
image_path = r"C:\Users\theod\Documents\UNI\thesis\DATA_tiles1\TILES_2.png"
output_dir = r"C:\Users\theod\Documents\UNI\thesis\DATA_tiles1\stick_crops_m2"
os.makedirs(output_dir, exist_ok=True)

crop_w, crop_h = 640, 1920  # output crop size
box_w, box_h = 222, 1766    # visual guide size

img = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_h, img_w = img.shape[:2]

clicks = []

# --- Mouse tracking ---
fig, ax = plt.subplots(figsize=(12, 12))
ax.imshow(img_rgb)
ax.set_title("Click stick centers. Press ENTER when done.")

cursor_box = ax.add_patch(
    plt.Rectangle((0, 0), box_w, box_h, linewidth=1.5, edgecolor='red', facecolor='none', visible=False)
)

def on_move(event):
    if event.xdata is not None and event.ydata is not None:
        cx, cy = int(event.xdata), int(event.ydata)
        cursor_box.set_visible(True)
        cursor_box.set_xy((cx - box_w / 2, cy - box_h / 2))
        fig.canvas.draw_idle()

def on_click(event):
    if event.xdata is not None and event.ydata is not None:
        cx, cy = int(event.xdata), int(event.ydata)
        clicks.append((cx, cy))
        ax.plot(cx, cy, 'go')
        print(f"🖱️ Clicked: ({cx}, {cy})")
        fig.canvas.draw_idle()

fig.canvas.mpl_connect('motion_notify_event', on_move)
fig.canvas.mpl_connect('button_press_event', on_click)
plt.show()

# --- Crop and save ---
for idx, (cx, cy) in enumerate(clicks):
    cx, cy = int(cx), int(cy)
    x1 = cx - crop_w // 2
    y1 = cy - crop_h // 2
    x2 = cx + crop_w // 2
    y2 = cy + crop_h // 2

    crop = np.zeros((crop_h, crop_w, 3), dtype=np.uint8)
    x1_img, y1_img = max(0, x1), max(0, y1)
    x2_img, y2_img = min(img_w, x2), min(img_h, y2)

    crop_x1, crop_y1 = x1_img - x1, y1_img - y1
    crop_x2 = crop_x1 + (x2_img - x1_img)
    crop_y2 = crop_y1 + (y2_img - y1_img)

    crop[crop_y1:crop_y2, crop_x1:crop_x2] = img_rgb[y1_img:y2_img, x1_img:x2_img]
    crop_bgr = cv2.cvtColor(crop, cv2.COLOR_RGB2BGR)

    out_path = os.path.join(output_dir, f"B_stick_{idx:03}.png")
    cv2.imwrite(out_path, crop_bgr)
    print(f" Saved: {out_path}")

print(f"\n Done! {len(clicks)} crops saved to: {output_dir}")



🎯 Done! 0 crops saved to: C:\Users\theod\Documents\UNI\thesis\DATA_tiles1\stick_crops_m2
